In [3]:
# in JupyterHub, plot difference in DT across globe between VIIRS and L3S
# HealPix
# once you can get geographic plots to work, everything else is just changing what you want to plot

In [2]:
import xarray
import numpy as np
import pandas
import h5py
import datetime
from ulmo.utils import image_utils
from ulmo.plotting import plotting
import matplotlib.pyplot as plt
import seaborn as sns
import os
import healpy as hp
# os.path.expanduser("~")

In [3]:
# read in 2012 h5 file

analysis_table = pandas.read_parquet('/home/jovyan/SST_L3S_VIIRS_2012.parquet')

file_path = '/home/jovyan/L3S_VIIRS144_2012_preproc.h5'
with h5py.File(file_path, 'r') as h5_file:
    h5_extractions = h5_file['valid']

In [32]:
analysis_table.columns[[20,21]]

Index(['lat', 'lon'], dtype='object')

In [3]:
analysis_table[['VIIRS_datetime', 'datetime']]

,VIIRS_datetime,datetime
0,2012-02-01 00:10:00,2012-01-31 13:42:26
1,2012-02-01 00:10:00,2012-01-31 13:44:47
2,2012-02-01 00:10:00,2012-01-31 13:49:19
3,2012-02-01 00:10:00,2012-01-31 13:46:48
4,2012-02-01 00:10:00,2012-01-31 13:44:00
...,...,...
293937,2012-12-31 23:20:00,2012-12-31 04:04:45
293938,2012-12-31 23:20:00,2012-12-31 03:48:18
293939,2012-12-31 23:20:00,2012-12-31 03:47:34
293940,2012-12-31 23:20:00,2012-12-31 03:46:52


In [4]:
file_path = '/home/jovyan/L3S_VIIRS144_2012_preproc.h5'

#with h5py.File(file_path, "r") as h5:
#       data = h5["valid"][:]

In [10]:
with h5py.File(file_path, "r") as h5:
        metadata = h5['valid_metadata'][:]

In [18]:
data1 = data[0]
metadata1 = metadata[0]

In [25]:
viirs_dt = float(metadata1[8])
dt = float(metadata1[35])
diff_dt = viirs_dt - dt

In [26]:
# row 9 is VIIRS_DT, row 36 is DT
diff_dt


0.972930908203125

In [5]:
with h5py.File(file_path, "r") as h5:
    metadata = h5["valid_metadata"][:]  # Replace "metadata" with the dataset name containing metadata

# Extract the 8th, 20th, 21st, and 35th indices for each cutout
cutout_indices = metadata[:, [8, 20, 21, 35]]

In [ ]:
# Loop through each cutout entry and plot the subarray area centered at lat, lon
nside = 64  # Adjust the nside to control the resolution of the map
for i in range(len(cutout_indices)):
    viirs_dt = float(cutout_indices[i, 0])
    lat = float(cutout_indices[i, 1])
    lon = float(cutout_indices[i, 2])
    dt = float(cutout_indices[i, 3])
    
    healpix_map = np.zeros(hp.nside2npix(nside))
    pix_index = hp.pixelfunc.ang2pix(nside, np.pi / 2 - np.radians(lat), np.radians(lon))
    healpix_map[pix_index] = viirs_dt - dt  
    hp.mollview(healpix_map, title=f"Cutout {i+1}", unit="Difference", min=-2, max=2, cmap='coolwarm')
    plt.close()

In [38]:
plt.show()

0.99853515625

In [ ]:


    # Create a Healpix map with the difference between viirs_dt and dt
    healpix_map = np.zeros(hp.nside2npix(nside))
    pix_index = hp.pixelfunc.ang2pix(nside, np.pi / 2 - np.radians(lat), np.radians(lon))
    healpix_map[pix_index] = viirs_dt - dt

    # Plot the subarray area centered at lat, lon using healpy
    hp.mollview(healpix_map, title=f"Cutout {i+1}", unit="Difference", min=-2, max=2, cmap='coolwarm')

plt.show()
In this code, we load the metadata from the h5 file, extract the 8th, 20th, 21st, and 35th indices for each cutout, and loop through each cutout entry. For each entry, we create a Healpix map (`




In [ ]:
#

# Assuming the datasets are in the format (number_of_cutouts, 64, 64)
# You may need to adjust the indices depending on the structure of your data
viirs_dt_july_20 = viirs_dt_data[:, :, :]
dt_july_20 = dt_data[:, :, :]

# Step 3: Compute the difference
difference = viirs_dt_july_20 - dt_july_20

# Step 4: Plot the difference across the globe using healpy
# Convert the difference to a Healpix map
nside = 64  # Adjust the nside to control the resolution of the map
healpix_map = hp.pixelfunc.ang2pix(nside, theta=np.pi / 2 - latitudes, phi=longitudes)

# Assuming you have latitude and longitude values for each cutout
# Replace latitudes and longitudes with the actual data
# Make sure to convert them to radians if they are in degrees

# Create the Healpix map with the difference values
hp_difference = np.zeros(hp.nside2npix(nside))
hp_difference[healpix_map] = difference

# Plot the difference map using healpy
hp.mollview(hp_difference, title="VIIRS_DT - DT Difference (July 20, 2020)", unit="YourUnit")
plt.show()

Plot the Healpix map
hp.mollview(map_data, cmap='jet', title='Ocean Plot')
hp.graticule()
plt.show()